In [1]:
import pandas as pd
import os
import datetime as dt

pd.set_option('display.max_columns', None)
#pd.reset_option('max_columns')

In [2]:
print(os.path.abspath('.'))

C:\Users\InfiniteWing\Downloads\assignment\stock_price


In [4]:
print("hello github!!")

hello github!


In [2]:
## 五檔報價 -----------------------------------------------------------------------------------------
raw_book_train = pd.read_parquet('assignment_600519/book/600519/20200401.parquet.gzip')
book_train = raw_book_train.copy()

In [56]:
book_train['TimeStamp2'] = pd.to_datetime(book_train['TimeStamp'].values, unit='us')
book_train.head()

,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,TimeStamp2
index,,,,,,,,,,,,,,,,,,,,,,,,
137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,2020-04-01 01:30:03.505567
138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,2020-04-01 01:30:06.961840
139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,2020-04-01 01:30:09.513298
140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,2020-04-01 01:30:12.124381
141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,2020-04-01 01:30:15.123840


In [57]:
## 時間
book_train['Week'] = book_train['TimeStamp2'].dt.dayofweek
book_train['Hour'] = book_train['TimeStamp2'].dt.hour
book_train['Minute'] = book_train['TimeStamp2'].dt.minute
book_train['Second'] = book_train['TimeStamp2'].dt.second
book_train.head()

,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,TimeStamp2,Week,Hour,Minute,Second
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,
137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,2020-04-01 01:30:03.505567,2,1,30,3
138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,2020-04-01 01:30:06.961840,2,1,30,6
139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,2020-04-01 01:30:09.513298,2,1,30,9
140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,2020-04-01 01:30:12.124381,2,1,30,12
141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,2020-04-01 01:30:15.123840,2,1,30,15


In [58]:
## 動能(漲跌相對強度)
col_bid = ['Bid1Size','Bid2Size','Bid3Size','Bid4Size','Bid5Size'] #買
col_ask = ['Ask1Size','Ask2Size','Ask3Size','Ask4Size','Ask5Size'] #賣

book_train['size_diff'] =  book_train[col_bid].sum(axis=1) - book_train[col_ask].sum(axis=1) #委買掛單總數-委賣掛單總數
book_train['size_ratio'] =  book_train[col_bid].sum(axis=1) / book_train[col_ask].sum(axis=1) #委買掛單總數/委賣掛單總數

book_train.head()

,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,TimeStamp2,Week,Hour,Minute,Second,size_diff,size_ratio
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,2020-04-01 01:30:03.505567,2,1,30,3,-458,0.955783
138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,2020-04-01 01:30:06.961840,2,1,30,6,-2340,0.719424
139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,2020-04-01 01:30:09.513298,2,1,30,9,-3940,0.543981
140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,2020-04-01 01:30:12.124381,2,1,30,12,2560,1.540084
141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,2020-04-01 01:30:15.123840,2,1,30,15,-3100,0.537313


In [59]:
## 交易量 -----------------------------------------------------------------------------------------
raw_trade_train = pd.read_parquet('assignment_600519/trade/600519/20200401.parquet.gzip')
trade_train = raw_trade_train.copy()

In [60]:
trade_train['TimeStamp2'] = pd.to_datetime(trade_train['TimeStamp'].values, unit='us')
trade_train.head()

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2
index,,,,,,,,
213,1585704600549504,99924,141932,0,0.0,100,1115.5,2020-04-01 01:30:00.549504
214,1585704600551309,142138,141795,100,1116.0,0,0.0,2020-04-01 01:30:00.551309
215,1585704601244086,86301,142421,0,0.0,100,1115.0,2020-04-01 01:30:01.244086
216,1585704601252795,141888,143205,0,0.0,100,1115.0,2020-04-01 01:30:01.252795
217,1585704601256878,141888,143923,0,0.0,200,1115.0,2020-04-01 01:30:01.256878


In [61]:
## 時間
trade_train['Week'] = trade_train['TimeStamp2'].dt.dayofweek
trade_train['Hour'] = trade_train['TimeStamp2'].dt.hour
trade_train['Minute'] = trade_train['TimeStamp2'].dt.minute
trade_train['Second'] = trade_train['TimeStamp2'].dt.second
trade_train.head()

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2,Week,Hour,Minute,Second
index,,,,,,,,,,,,
213,1585704600549504,99924,141932,0,0.0,100,1115.5,2020-04-01 01:30:00.549504,2,1,30,0
214,1585704600551309,142138,141795,100,1116.0,0,0.0,2020-04-01 01:30:00.551309,2,1,30,0
215,1585704601244086,86301,142421,0,0.0,100,1115.0,2020-04-01 01:30:01.244086,2,1,30,1
216,1585704601252795,141888,143205,0,0.0,100,1115.0,2020-04-01 01:30:01.252795,2,1,30,1
217,1585704601256878,141888,143923,0,0.0,200,1115.0,2020-04-01 01:30:01.256878,2,1,30,1


In [66]:
trade_train.head(35)

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2,Week,Hour,Minute,Second
index,,,,,,,,,,,,
213,1585704600549504,99924,141932,0,0.00,100,1115.50,2020-04-01 01:30:00.549504,2,1,30,0
214,1585704600551309,142138,141795,100,1116.00,0,0.00,2020-04-01 01:30:00.551309,2,1,30,0
215,1585704601244086,86301,142421,0,0.00,100,1115.00,2020-04-01 01:30:01.244086,2,1,30,1
216,1585704601252795,141888,143205,0,0.00,100,1115.00,2020-04-01 01:30:01.252795,2,1,30,1
217,1585704601256878,141888,143923,0,0.00,200,1115.00,2020-04-01 01:30:01.256878,2,1,30,1
218,1585704602351479,145463,142415,100,1115.50,0,0.00,2020-04-01 01:30:02.351479,2,1,30,2
219,1585704602351488,145463,142417,100,1115.50,0,0.00,2020-04-01 01:30:02.351488,2,1,30,2
220,1585704602351495,145463,142975,100,1115.50,0,0.00,2020-04-01 01:30:02.351495,2,1,30,2
221,1585704602361905,141888,146661,0,0.00,200,1115.00,2020-04-01 01:30:02.361905,2,1,30,2
